<a href="https://colab.research.google.com/github/OnishchenkoDaria/numeric-methods-lab1/blob/main/numeric_methods_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import numpy as np
import math

#define the default function
def function(x):
    return (pow(x, 3) - 3 * pow(x, 2) - 17 * x + 22)

#define the first derivative of the function
def derivative_f(x):
    return (3 * pow(x, 2) - 6 * x - 17)

#define the second derivative of the function
def sec_derivative_f(x):
    return (6 * x - 6)

#process user input
def get_user_input(accuracy_def, x0_def):
    #setting interval
    interval_instruction = "Pass the interval to be reviewed in form of 0,1 or press Enter to leave as default: "
    interval_input = input(interval_instruction)
    interval = tuple(map(float, interval_input.split(","))) if interval_input else (1, 2)

    #setting accuracy
    accuracy_instruction = "Pass the accuracy to be reviewed in form of 1e-4 or press Enter to leave as default: "
    accuracy_input = input(accuracy_instruction)
    accuracy = float(accuracy_input) if accuracy_input else accuracy_def

    #setting the initial point
    x0_instruction = f"Pass the point approximate step to be reviewed in form of {x0_def} or press Enter to leave as default: \n"
    x0_input = input(x0_instruction)
    x0 = float(x0_input) if x0_input else x0_def
    return interval, accuracy, x0

#checking Newton method solution conditions
def newton_check(x_min, x_max):
    if function(x_min) * function(x_max) < 0:  # Ensure a root exists between x_min and x_max
        # Check if the second derivative is positive across the range
        x_vals = np.linspace(x_min, x_max, 50)
        sec_derivative_vals = np.array([sec_derivative_f(x) for x in x_vals])
        if (np.all(sec_derivative_vals >= 0) or np.all(sec_derivative_vals <= 0)):
            return True
        else:
            return False

def calculate_interval(interval, x0):
    x_min, x_max = interval
    x_avg = (x_max + x_min) / 2

    if function(x_avg) < 0 and function(x_min) < 0:
        x_min = x_avg
    else:
        x_max = x_avg

    print(f"min: {x_min}, max: {x_max}")

    if(x0 > x_max or x0 < x_min):
      x0 = (x_min + x_max) / 2
      print(f"\nx0 is out of range, changing it to {x0}")

    return x_min, x_max, x0

#define the optimal value of t and other metrics
def calculate_metrics(x_min, x_max, x0):
    #take 100 points between min and max
    x_values = np.linspace(x_min, x_max, 50)

    #evaluate the derivative at each x_value
    derivative_arr = np.array([derivative_f(x) for x in x_values])
    sec_derivative_arr = np.array([sec_derivative_f(x) for x in x_values])

    #get the min and max values of the absolute derivative
    m1 = min(np.abs(derivative_arr))
    M1 = max(np.abs(derivative_arr))
    M2 = max(np.abs(sec_derivative_arr))  #for newton's method

    #calculate optimal t for the relaxation method
    t_opt = 2 / (m1 + M1)

    #initial z0 (distance between max and min estimate)
    z0 = 0
    for x in x_values:
      if np.abs(x0 - x) > z0:
        z0 = np.abs(x0 - x)

    return t_opt, m1, M1, z0, M2

#calculate the number of iterations based on method type
def calculate_iterations(z0, accuracy, q, method_type):
    primary_calc = math.log(abs(z0 / accuracy)) / math.log(1 / q)
    if method_type == "relax":
        return math.ceil(primary_calc + 1)
    elif method_type == "newton":
        print(primary_calc)
        return math.ceil(math.log2(primary_calc + 1))

#relaxation method
def relaxation_method(function, derivative_f, x0, accuracy, t, max_iterations=1000):
    x = x0
    x_vals = []
    x_vals.append(x0)

    for i in range(max_iterations):
        fx = function(x)
        derivative_f_x = derivative_f(x)

        #determine the direction based on the derivative sign
        if derivative_f_x > 0:
            x_new = x - t * fx
        else:
            x_new = x + t * fx

        #if the change is smaller than the desired accuracy, stop
        if abs(x_new - x) < accuracy:
            x_vals.append(x_new)
            return x_vals

        #update x and append the new value
        x_vals.append(x_new)
        x = x_new

    #if the loop completes without converging
    print("Failed to converge within the maximum number of iterations")
    return x_vals

#newton method
def newton_method(function, derivative_f, x0, newton_check, accuracy, max_iterations=1000):
    x = x0
    x_vals = []
    x_vals.append(x0)

    #checking the passed newton conditions

    if newton_check == True:
      if(function(x0)*sec_derivative_f(x0) > 0):

        for i in range(max_iterations):
            #checking the second derivative condition on the interval
            if derivative_f(x) == 0:
                print("Derivative became zero.")
                return x_vals

            #calculating x_n
            x_new = x - function(x) / derivative_f(x)

            #if the change is smaller than the desired accuracy, stop
            if abs(x_new - x) < accuracy:
                x_vals.append(x_new)
                return x_vals

            #update x and append the new value
            x_vals.append(x_new)
            x = x_new

        #if the loop completes without converging
        print("Failed to converge within the maximum number of iterations")
        return x_vals

      else:
        print("Failed checking Newton conditions")
        return x_vals

#print the table of results
def print_table(x_vals, n_iterations, q, m1, M1, M2):
    print(f"\n{'Index':<10} {'x value':<20} {'Accuracy':<20}")
    print("=" * 50)

    for i in range(len(x_vals)):
        x = x_vals[i]
        if i > 0:
            e = abs(x_vals[i] - x_vals[i - 1])
        else:
            e = x_vals[i]
        print(f"{i:<10} {x:<20} {e:.16f}")

    #output results
    print(f"\nm1 (min abs derivative): {m1}")
    print(f"M1 (max abs derivative): {M1}")
    print(f"M2 (max abs sec derivative): {M2}")
    print(f"q (convergence factor): {q:.4f}")
    print(f"n_iterations estimated prediction: {n_iterations}")

#handling all the metrics and functions called for relaxation
def solve_with_relaxation(interval, x0):
    print("\nRELAXATION METHOD\n")

    x_min, x_max, x0 = calculate_interval(interval, x0)
    t_opt, m1, M1, z0, M2 = calculate_metrics(x_min, x_max, x0)

    q = (M1 - m1) / (m1 + M1)

    if(q > 0):
      n_iterations_relax = calculate_iterations(z0, accuracy, q, "relax")
      x_vals_relax = relaxation_method(function, derivative_f, x0, accuracy, t_opt)
      print_table(x_vals_relax, n_iterations_relax, q, m1, M1, M2)
      print(f"z0 (absoltue x0 and x diff): {z0:.2f}")
      print(f"Optimal t for relaxation: {t_opt:.4f}")
    else:
      print("q check failed: less than zero")

#handling all the metrics and functions called for newton
def solve_with_newton(interval, x0):
    print("\nNEWTON METHOD\n")

    x_min, x_max, x0 = calculate_interval(interval, x0)
    t_opt, m1, M1, z0, M2 = calculate_metrics(x_min, x_max, x0)

    if(abs(x_max - x0) > abs(x_min -x0)): #delta for newton
        delta = abs(x_max - x0)
    else:
        delta = abs(x_min - x0)

    check = newton_check(x_min, x_max)
    print(check)

    q = (M2 * delta) / (2 * m1)

    if(q < 1):
      n_iterations_newton = calculate_iterations(delta, accuracy, q, "newton")
      x_vals_newton = newton_method(function, derivative_f, N_x0, check, accuracy)
      print_table(x_vals_newton, n_iterations_newton, q, m1, M1, M2)
      print(f"delta: {delta:.2f}")
    else:
      print("q check failed: less than zero")

if __name__ == "__main__":
    #relaxation method
    interval, accuracy, x0 = get_user_input(1e-4, 1.25)

    #newton method
    N_interval, N_accuracy, N_x0 = get_user_input(1e-4, 1.15)

    #relaxation method
    solve_with_relaxation(interval, x0)

    #newton method
    solve_with_newton(N_interval, N_x0)


Pass the interval to be reviewed in form of 0,1 or press Enter to leave as default: -4,-3
Pass the accuracy to be reviewed in form of 1e-4 or press Enter to leave as default: 
Pass the point approximate step to be reviewed in form of 1.25 or press Enter to leave as default: 
-3.75
Pass the interval to be reviewed in form of 0,1 or press Enter to leave as default: -4,-3
Pass the accuracy to be reviewed in form of 1e-4 or press Enter to leave as default: 
Pass the point approximate step to be reviewed in form of 1.15 or press Enter to leave as default: 
-3.75

RELAXATION METHOD

min: -4.0, max: -3.5

Index      x value              Accuracy            
0          -3.75                -3.7500000000000000
1          -3.558420365535248   0.1915796344647518
2          -3.5468923455104577  0.0115280200247905
3          -3.545521040368555   0.0013713051419026
4          -3.545352874286055   0.0001681660825001
5          -3.5453321779726705  0.0000206963133844

m1 (min abs derivative): 40.75
M1